In [1]:
!pip install altair

In [2]:
!pip install jupyter_bokeh

In [3]:
!pip install vega_datasets

In [4]:
import panel as pn
import altair as alt
import pandas as pd
from vega_datasets import data
import datetime as dt
from altair import datum

In [5]:
alt.renderers.enable('default')
pn.extension('vega')

In [6]:
source = data.stocks()
source = pd.DataFrame(source)
source.head()

,symbol,date,price
0,MSFT,2000-01-01,39.81
1,MSFT,2000-02-01,36.35
2,MSFT,2000-03-01,43.22
3,MSFT,2000-04-01,28.37
4,MSFT,2000-05-01,25.45


In [7]:
# Dashboard title
title = '## Stock Price Dashboard'

# Dashboard subtitle
subtitle = 'This dashboard allows selection of company and date range.'

# List of company tickers
tickers = ['AAPL', 'AMZN', 'GOOG', 'IBM', 'MSFT']

# Dropdown widget for company selection
ticker = pn.widgets.Select(name='Company', options=tickers)

# Date range slider
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range',
    start=dt.datetime(2001, 1, 1), end=dt.datetime(2010, 1, 1),
    value=(dt.datetime(2001, 1, 1), dt.datetime(2010, 1, 1))
)

# Define a color map for each ticker
ticker_colors = {
    'AAPL': 'red',
    'AMZN': 'blue',
    'GOOG': 'green',
    'IBM': 'purple',
    'MSFT': 'orange'
}

# Function to update the plot data based on selected ticker and date range
@pn.depends(ticker.param.value, date_range_slider.param.value)
def get_plot(ticker, date_range):
    df = source.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Filter data by date range and selected ticker
    start_date = pd.to_datetime(date_range[0])
    end_date = pd.to_datetime(date_range[1])
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date) & (df['symbol'] == ticker)]

    # Get the color for the selected ticker
    line_color = ticker_colors.get(ticker, 'gray')

    # Create the Altair chart with the filtered data and set the line color
    chart = alt.Chart(filtered_df).mark_line(color=line_color).encode(
        x='date',
        y='price',
        tooltip=['date', 'price', 'symbol']
    )

    return chart

# Create the dashboard layout
dashboard = pn.Row(
    pn.Column(title, subtitle, ticker, date_range_slider),
    get_plot
)

In [8]:
import warnings
from bokeh.util.warnings import BokehUserWarning

# Filter out the specific BokehUserWarning
warnings.filterwarnings(
    "ignore",
    message="reference already known",
    category=BokehUserWarning
)

dashboard.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Select(name='Company', options=['AAPL', 'AMZN', ...], value='AAPL')
        [3] DateRangeSlider(end=datetime.datetime(2010, ..., name='Date Range', start=datetime.datetime(2001, ..., value=(datetime.datetime(2001, ..., value_end=datetime.datetime(2010, ..., value_start=datetime.datetime(2001, ...)
    [1] ParamFunction(function, _pane=Vega, defer_load=False)